### Enter full names of group members:

##### Name A: Vegard Vaeng Bernhardsen
##### Name B: None

In [1]:
import math
import numpy as np
import sympy
from pathlib import Path  # for paths of files
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity


# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [2]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [3]:
def initialize_buckets():
    """Initialize an empty list to store buckets."""
    return []

def update_timestamp(timestamp, N):
    """Update the timestamp to simulate a circular buffer."""
    return (timestamp + 1) % N

def remove_old_buckets(buckets, timestamp, N):
    """Remove buckets that are outside the sliding window."""
    return [bucket for bucket in buckets if (timestamp - bucket[1]) % N < N - 1]

def merge_buckets(buckets):
    """Merge consecutive buckets of the same size."""
    i = 0
    while i < len(buckets) - 2:
        if buckets[i][0] == buckets[i + 1][0] == buckets[i + 2][0]:
            new_size = buckets[i][0] * 2
            new_timestamp = buckets[i + 1][1]
            buckets[i + 1] = (new_size, new_timestamp)
            del buckets[i]
        else:
            i += 1

def dgim_algorithm(stream_path, N):
    """Implementation of the DGIM algorithm."""
    buckets = initialize_buckets()
    timestamp = 0

    with open(stream_path) as f:
        while True:
            bit = f.read(1)
            if not bit:
                break

            timestamp = update_timestamp(timestamp, N)

            buckets = remove_old_buckets(buckets, timestamp, N)

            if bit == '1':
                buckets.append((1, timestamp))
                merge_buckets(buckets)

    # Prepare output
    bucket_list = [[] for _ in range(math.ceil(math.log2(N)))]
    end_time_stamp = None

    for size, ts in buckets:
        index = int(math.log2(size))
        if index < len(bucket_list):
            bucket_list[index].append(ts)
            if size == 1:
                end_time_stamp = ts

    for blist in bucket_list:
        blist.sort()

    return bucket_list, end_time_stamp


bucket, end_time_stamp = dgim_algorithm(stream_path, N)
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket}")
print(f"The end timestamp: {end_time_stamp}")


The updated list of timestamps buckets from DGIM algorithm: 
 [[99], [91, 96], [83, 89], [63, 75], [44], [6], [321, 446], [188], []]
The end timestamp: 99


In [4]:
bucket = dgim_algorithm(stream_path, N)

In [5]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [[99], [91, 96], [83, 89], [63, 75], [44], [6], [321, 446], [188], []]
The end timestamp: 99


#### 1.2. Query the Bucket 

In [6]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [7]:
def get_effective_count(bucket_list, end_time_stamp, N, k):
    """Calculate the estimated number of 1s in the last k bits based on DGIM buckets."""
    one_count = 0
    last_bucket = 0

    # Calculate contributions of buckets within the window
    for size, timestamps in enumerate(bucket_list):
        bucket_size = 2 ** size
        for timestamp in timestamps:
            if (end_time_stamp - timestamp) % N < k:
                one_count += bucket_size
                last_bucket = size

    # Adjust by subtracting half the size of the last bucket to reduce overcount
    if last_bucket:
        one_count -= 2**(last_bucket) / 2

    return math.ceil(one_count)

def dgim_query(bucket, N, k):
    """Query the DGIM algorithm for the count of 1s in the last k bits."""
    bucket_list, end_time_stamp = bucket
    return get_effective_count(bucket_list, end_time_stamp, N, k)


In [8]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [9]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [10]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [11]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [12]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [13]:
def generate_hash(h, N):
    hash_list = []
    used_primes = set()
    
    for _ in range(h):
        random_prime = generate_unique_prime(used_primes)
        hash_function = create_hash_function(random_prime, N)
        hash_list.append(hash_function)
    
    return hash_list

def generate_unique_prime(used_primes):
    while True:
        random_prime = sympy.randprime(1, 10000000)
        if random_prime not in used_primes:
            used_primes.add(random_prime)
            return random_prime

def create_hash_function(prime_number, N):
    def hash_function(s):
        if prime_number is not None:
            return sum((ord(c) * pow(prime_number, i, N) for i, c in enumerate(s))) % N
        else:
            raise ValueError("Prime number cannot be None")
    return hash_function


In [14]:
hashes = generate_hash(h, bloom_size)

In [15]:
def create_bloom_filter(B, hashes, data):
    with data.open(encoding='utf-8') as f:  # Specify encoding as 'utf-8'
        for name in f:
            name = name.strip()  # Remove leading/trailing whitespaces
            for hash_func in hashes:
                index = hash_func(name)
                B[index] = 1  # Set the corresponding bit in the Bloom filter array to 1
    return B


In [16]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [17]:
bloom_array

array([0., 1., 1., ..., 0., 0., 0.])

#### 2.2. Verify usernames

In [18]:
def single_verify_username(bloom_array, hashes, new_user):
    for hash_func in hashes:
        index = hash_func(new_user)
        if bloom_array[index] == 0:
            return 0  # Username is available
    return 1  # Username is likely taken
    

In [19]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"

In [20]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [21]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username KazeemTDT4305 has been taken. Try again!


In [22]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open(encoding='utf-8') as f:
        for name in f:
            name = name.strip()  # Remove leading/trailing whitespaces
            total_name += 1
            taken_name += single_verify_username(bloom_array, hashes, name)
            
    return round(taken_name/total_name*100,2)   

In [23]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------


Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.67%
----------------------------------------------------------


### 3. Flajolet-Martin

In [24]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # Define hash function h(x) = 6x + 1 mod 5
    def hash_function(x):
        return (6 * x + 1) % 5

    # Iterate over the input stream and update maximum rightmost zero bit position
    for element in input_stream:
        hash_value = hash_function(element)
        binary_rep = bin(hash_value)[2:]  # Convert hash value to binary representation
        rightmost_zero_bit = len(binary_rep) - binary_rep.rfind('0') - 1
        if rightmost_zero_bit > R:
            R = rightmost_zero_bit

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [25]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 4
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [26]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [27]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [28]:

def greedy_algorithm(companies, queries):
    revenue = 0
    
    for query in queries:
        potential_company = []

        for company, i in companies.items():
            keywords, budget = i[:-1], i[-1]
            
            if query in keywords and budget > 0:
                potential_company.append(company)
            
        if potential_company:
            chosen_bidder = random.choice(potential_company)
            revenue += 1  # Increment revenue by 1 for each query-advertiser match

            companies[chosen_bidder][-1] -= 1
    
    return revenue


In [29]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 11
Trial 2 - Revenue generated: 10
Trial 3 - Revenue generated: 9
Trial 4 - Revenue generated: 11
Trial 5 - Revenue generated: 8
Trial 6 - Revenue generated: 10
Trial 7 - Revenue generated: 10
Trial 8 - Revenue generated: 10
Trial 9 - Revenue generated: 8
Trial 10 - Revenue generated: 10
------------------------------------------------
Average revenue generated for all trials:  9.7


#### 4.2. Balance Algorithm

In [30]:
def balance_algorithm(companies, queries):
    revenue = 0
    
    for query in queries:
        potential_advertisers = []

        # Collect all advertisers with matching keywords and positive budget
        for company, data in companies.items():
            keywords, budget = data[:-1], data[-1]
            if query in keywords and budget > 0:
                potential_advertisers.append((company, budget))

        # If there are potential advertisers, proceed with selection
        if potential_advertisers:
            # Sort advertisers by budget, descending, and find the highest budget
            potential_advertisers.sort(key=lambda x: x[1], reverse=True)
            highest_budget = potential_advertisers[0][1]

            # Filter for advertisers with the highest budget
            highest_budget_advertisers = [adv[0] for adv in potential_advertisers if adv[1] == highest_budget]

            # Randomly select one advertiser from those with the highest budget
            chosen_bidder = random.choice(highest_budget_advertisers)
            revenue += 1  # Increment revenue for each match

            # Decrement the budget of the chosen advertiser
            companies[chosen_bidder][-1] -= 1
    
    return revenue

In [31]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 9
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 8
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 8
Trial 6 - Revenue generated: 10
Trial 7 - Revenue generated: 9
Trial 8 - Revenue generated: 10
Trial 9 - Revenue generated: 10
Trial 10 - Revenue generated: 9
-------------------------------------------
Average revenue generated for all trials:  9.1


### 5. Recommender System

In [32]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [33]:
def get_user_ratings(rate_m, user_index):
    """Get ratings for a specific user."""
    return rate_m[:, user_index].reshape(1, -1)

def compute_similarity(r_x, r_y):
    """Compute cosine similarity between two rating vectors."""
    return cosine_similarity(r_x, r_y)[0][0]

def user_cf(rate_m, tup_mu, neigh):
    """User-User Collaborative Filtering using cosine similarity."""
    # Extract movie index and user index
    movie_index, user_index = tup_mu[0] - 1, tup_mu[1] - 1
    
    # Get ratings for the target user
    r_x = get_user_ratings(rate_m, user_index)

    # Compute similarity between the target user and all other users
    most_similar = []
    for i in range(len(rate_m[0])):
        if i == user_index:
            continue
        
        r_y = get_user_ratings(rate_m, i)
        sim = compute_similarity(r_x, r_y)
        most_similar.append((i, sim))
    
    # Sort users by similarity and limit to the specified neighborhood
    most_similar = sorted(most_similar, key=lambda x: x[1], reverse=True)[:neigh]

    # Compute weighted sum of ratings and total similarity
    weighted_ratings, total_sim = 0, 0
    for user, sim in most_similar:
        movie_rating = rate_m[movie_index, user]
        weighted_ratings += sim * movie_rating
        total_sim += sim
    
    # Handle division by zero
    if total_sim == 0:
        return 0
    
    # Compute the predicted rating
    prediction = round(weighted_ratings / total_sim, 2)
    
    return prediction


In [34]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [35]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.42 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.49 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [36]:
def get_movie_ratings(rate_m, movie_index):
    """Get ratings for a specific movie."""
    return rate_m[movie_index, :].reshape(1, -1)

def compute_similarity(r_x, r_y):
    """Compute cosine similarity between two rating vectors."""
    return cosine_similarity(r_x, r_y)[0][0]

def item_cf(rate_m, tup_mu, neigh):
    """Item-Item Collaborative Filtering using cosine similarity."""
    # Extract movie index and user index
    movie_index, user_index = tup_mu[0] - 1, tup_mu[1] - 1
    
    # Get ratings for the target movie
    r_x = get_movie_ratings(rate_m, movie_index)

    # Compute similarity between the target movie and all other movies
    most_similar = []
    for i in range(len(rate_m)):
        if i == movie_index:
            continue
        
        r_y = get_movie_ratings(rate_m, i)
        sim = compute_similarity(r_x, r_y)
        most_similar.append((i, sim))
    
    # Sort movies by similarity and limit to the specified neighborhood
    most_similar = sorted(most_similar, key=lambda x: x[1], reverse=True)[:neigh]

    # Compute weighted sum of ratings and total similarity
    weighted_ratings, total_sim = 0, 0
    for movie, sim in most_similar:
        movie_rating = rate_m[movie, user_index]
        weighted_ratings += sim * movie_rating
        total_sim += sim
    
    # Handle division by zero
    if total_sim == 0:
        return 0
    
    # Compute the predicted rating
    prediction = round(weighted_ratings / total_sim, 2)
    
    return prediction


In [37]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

It's true the number of buckets used in DGIM scales scales logarthmically with the size of the sliding window N, the actual space complexity is determined by the amount of memory each bucket consumes. 
Each bucket stores O(log n) bits for the timestamp. So there are O(log n) buckets wich all store O(log n) bits. Thus the space complexity of the DGIM algorithm is O(log^2 n).


#### Case 2

If the bloom filter outputs that "Kazeem" is taken then their is a certain possibility that this is a false positive. This is because with bloom filters, although with create memory usage, it could give out false positives. I would additionally tell the admin that this is highly correlated to the use of hash functions. It is possible that somehow all the hash functions consider a username to be taken, due to imperfected hashes. 

In this case the bloom filter outputs that a username has NOT been taken. A bloom filter guarantees no false negatives, hence it is impossible that the username "KazeemTDT4305" has been taken. For the bloom filter to return that a username has NOT been taken, only one of the indexes given by one hash function in the bloom filter must be 0. This will not happen if the username was taken. 

The reason that Google finds that particular email to be taken could be because of false positives when using bloom fitlers as discussed.

#### Case 3

To improve precision with the Flajolet-Martin algorithm, we employ multiple hash functions grouped into sets. Instead of relying on just one hash function, using many allows us to gather diverse data from the input stream, reducing the chance of errors. Each group of hash functions works independently, providing unique insights into the data. Within each group, we choose the median value to minimize the impact of outliers. Finally, we calculate the average of all median values obtained from different groups, resulting in a more accurate estimate of the distinct count. By following this method, we enhance the accuracy of the Flajolet-Martin algorithm's estimates, making it more reliable for estimating the number of unique elements in a data stream.








#### Case 4

# Performance Analysis of Greedy and Balance Algorithms

**Maximum Possible Revenue**

For maximum revenue:
- **Company D** handles all queries for 'big data', earning **3 dollars**. Company D's budget is then depleted.
- **Company A** addresses all queries for 'bloom filters', adding **3 dollars** for a total of **6 dollars**. Company A's budget is used up.
- **Company B** covers all queries for 'flajolet martin', increasing the total to **9 dollars**. Company B's budget is spent.
- **Company C** manages all queries for 'dgim algorithm', reaching a total revenue of **12 dollars**. Company C's budget is exhausted.

This optimal distribution results in a total revenue of **12 dollars**.

**Minimum Possible Revenue**

For the minimum revenue:
- **Company A** takes all queries for 'big data', earning **3 dollars**. Company A's budget is depleted.
- **Company C** takes all queries for 'flajolet martin', making the total revenue **6 dollars**. Company C's budget is spent.

The remaining queries ('bloom filters' and 'dgim algorithm') do not match the interests of the remaining companies with available budgets, resulting in a minimum revenue of **6 dollars**.

**Competitive Ratio for the Greedy Algorithm**

The Greedy algorithm could randomly pick bidders that lead to this minimum revenue scenario:
- By potentially picking advertisers in a manner that matches the minimum revenue scenario, the Greedy algorithm's worst performance would be a revenue of **6 dollars**.

The competitive ratio thus becomes:

Competitive ratio (Greedy) = 1/2

**Competitive Ratio for the Balance Algorithm**

In the worst case for the Balance algorithm:
- **Company A** takes 2 queries for 'big data' and 1 for 'bloom filter', earning **3 dollars**.
- **Company B** takes 1 query for 'flajolet martin', contributing **1 dollar**.
- **Company C** addresses 2 queries for 'flajolet martin' and 1 for 'dgim algorithm', adding **3 dollars**.
- **Company D** handles 1 query for 'big data', adding **1 dollar**.

This yields a total revenue of **8 dollars**.

The competitive ratio thus becomes:

Competitive ratio (Balance) = 2/3



#### Case 5

Upon analyzing the ratings given by user 5, we observe that their average rating across all movies is **3.5**. Comparing this to the predictions made by User-User and Item-Item Collaborative Filtering for movie 1, the User-User CF predicts a rating of **1.42**, while the Item-Item CF predicts a rating of **2.42**. Given that the Item-Item CF's prediction is closer to user 5's average rating, it suggests that this method might provide a more accurate estimation in this case.

Furthermore, evaluating the overall quality of movie 1 based on its average rating (which is **4.5**), we can deduce that movie 1 is generally received well and not a poorly rated movie. This context reinforces the idea that the higher prediction (**2.42**) from Item-Item CF aligns better with the general perception of the movie. Therefore, for this scenario, Item-Item CF appears to offer a more reliable and realistic prediction, capturing both the trends in user 5’s ratings and the movie’s general appeal more effectively.
